# ResNet-BK Benchmarking

Compare different configurations and measure performance.

This notebook benchmarks:
- Baseline vs optimized configurations
- Different GRAD_BLEND values
- Model size scaling
- Sequence length scaling

In [ ]:
!pip install datasets torch matplotlib pandas -q

In [ ]:
# Repo setup (clone if needed, add to sys.path)
import os, sys, subprocess, pathlib
REPO_URL = 'https://github.com/neko-jpg/Project-ResNet-BK-An-O-N-Language-Model-Architecture.git'
REPO_DIR = 'Project-ResNet-BK-An-O-N-Language-Model-Architecture'
cwd = pathlib.Path.cwd()
candidates = [cwd, cwd.parent, cwd / REPO_DIR, cwd.parent / REPO_DIR]
root = next((p for p in candidates if (p / 'src').exists()), None)
if root is None:
    root = cwd / REPO_DIR
    if not root.exists():
        subprocess.run(['git', 'clone', REPO_URL, str(root)], check=True)
if root != pathlib.Path.cwd():
    os.chdir(root)
root_str = str(pathlib.Path.cwd())
if root_str not in sys.path:
    sys.path.insert(0, root_str)
print('PWD:', root_str)


In [ ]:
import torch
import pandas as pd
import matplotlib.pyplot as plt
from src.models.configurable_resnet_bk import (
    ConfigurableResNetBK,
    BASELINE_CONFIG,
    STEP2_CONFIG,
    FULL_CONFIG
)
from src.utils import get_data_loader

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"Using device: {device}")

## Benchmark Different Configurations

In [ ]:
configs = {
    'Baseline': BASELINE_CONFIG,
    'Step 2': STEP2_CONFIG,
    'Full': FULL_CONFIG,
}

results = []

for name, config in configs.items():
    print(f"\nBenchmarking {name}...")
    
    model = ConfigurableResNetBK(config).to(device)
    summary = model.get_config_summary()
    
    print(f"Parameters: {summary['Parameters']}")
    print(f"Features: {summary['Enabled Features']}")
    
    results.append({
        'Config': name,
        'Parameters': summary['Parameters'],
        'Features': len(summary['Enabled Features']),
    })

df = pd.DataFrame(results)
print("\n", df)

## Visualize Results

In [ ]:
fig, ax = plt.subplots(figsize=(10, 6))
ax.bar(df['Config'], df['Features'])
ax.set_xlabel('Configuration')
ax.set_ylabel('Number of Enabled Features')
ax.set_title('ResNet-BK Configuration Comparison')
plt.tight_layout()
plt.show()